# Agentics Tutorial
This is a step by step guide illustrating how to make a new AG, access and print its content, import and export it to files

In [ ]:
! uv pip install agentics-py

import os
from pathlib import Path
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)

if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive

    drive.mount("/content/drive")
    from google.colab import userdata

    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

base = Path(CURRENT_PATH)

## Define a new AG

Agentics provide the smarts of LLM to pydantic types. It does so by wrapping assigning any Pydantic type definitions to the atype field. 

In [ ]:
from agentics import AG
from pydantic import BaseModel
from typing import Optional


class Movie(BaseModel):
    title: Optional[str] = None
    genre: Optional[str] = None
    description: Optional[str] = None


movies = AG(atype=Movie)
print(movies.pretty_print())

## Agentics are Typed Lists

In addition to atypes, the AG class allows us to store a list of pydantic objects which are instances of atype. This enable them to exibit a list like behaviour, which allows us to access their states as arrays and use the + operator for concatenation.


In [ ]:
movies = AG(atype=Movie)
print(movies.states)  ### Initially empty states

movies.states.append(Movie(title="The Godfather"))  ### add new object
print(movies[0])  ## print the first state

movies.states.append(Movie(title="La dolce vita"))  ## add second object
print(
    movies.pretty_print()
)  ## the pretty print function return a string describing atype and states of the agentic

for movie in movies:  ### Agentics can be used as lists and allow iteration on states
    print(movie.model_dump_json())

new_movies = AG(
    atype=Movie, states=[Movie(title="Superman")]
)  ### Creating a new agentics to show sum


for state in new_movies:
    movies.append(state)

print(movies.pretty_print())

print(len(movies))

## Input/Output

Agentics states can be initialized loaded and saved to .csv or .jsonl files. AType will be automatically generated from the column names (after they will be normalized as needed for valid pydantic fields), with all attributes set to strings.


In [ ]:
# Create a new AG object from the privided csv file
movies = AG.from_csv(base / "data/movies.csv", max_rows=10)

# Note that the type is automatically induced and all arguments are set to srt
print(movies.pretty_print())

# Export the same AG in jsonl format
movies.to_jsonl(base / "data/movies.jsonl")

# Load new agentic from jsonl
movies_copy = AG.from_jsonl(base / "data/movies.jsonl")
for state in movies:
    movies_copy.append(state)

# export the new AG in csv format
movies_copy.to_csv(base / "data/movies_copy.csv")